In [1]:
import pandas as pd

from autogluon.tabular import TabularPredictor
from autogluon.core.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc

In [2]:
def pr_auc_Metric(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    score = auc(recall, precision)
    return score

In [3]:
ag_roc_auc_scorer = make_scorer(
    name='roc_auc',
    score_func=pr_auc_Metric,
    optimum=1,
    greater_is_better=True,
    needs_proba=True
)

In [4]:
TIME_LIMIT = 2 * 3600

In [5]:
extra_metrics = ['roc_auc', 'accuracy', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [6]:
for i in range(0, 10):
    print(f'Fitting model {i}')
    original_data = pd.read_csv(
        f'../../data/train/set_{i}.csv'
        )
    synthetic_data = pd.read_csv(
        f'../../data/synthetic/tvaes/set_{i}.csv'
    )
    # Conteo de clases
    original_vc = original_data['ED_2Clases'].value_counts()
    synthetic_vc = synthetic_data['ED_2Clases'].value_counts()
    print(f"Original data: {original_vc['D']} {original_vc['H']}")
    print(f"Synthetic data: {synthetic_vc['D']} {synthetic_vc['H']}")

    # Calcular cuántos 'D' sintéticos tomar
    d_sample = original_vc['H'] + synthetic_vc['H'] - original_vc['D']
    print(f"Sampled data: {d_sample}")

    # Seleccionar datos sintéticos
    synth_sampled = pd.concat([
        synthetic_data[synthetic_data['ED_2Clases'] == 'H'],
        synthetic_data[synthetic_data['ED_2Clases'] == 'D'].sample(d_sample, random_state=42)
    ])

    # Combinar y mezclar
    train_data = pd.concat([original_data, synth_sampled], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Train data: {train_data['ED_2Clases'].value_counts()}")

    predictor= TabularPredictor(
        label='ED_2Clases',
        problem_type='binary',
        eval_metric=ag_roc_auc_scorer,
        sample_weight='balance_weight',
        path=f'AutogluonModels/mix/v{i}'
    )

    predictor.fit(
        train_data = train_data,
        time_limit = TIME_LIMIT,
        presets = ['high_quality'],
        auto_stack = False,
        excluded_model_types=['KNN', 'LR'],
        num_stack_levels=0,
        num_bag_folds=0,
        verbosity = 2
    )

    df_test = pd.read_csv(
        f'../../data/test/set_{i}.csv'
    )

    predictors = predictor.leaderboard(
        df_test,
        extra_metrics = extra_metrics,
        extra_info=True,
        silent=True
    )
    predictors.to_excel(
        f'../../5_results/5_3_mix/predictors_set_{i}.xlsx',
        index=False
    )

    feature_importance = predictor.feature_importance(train_data)
    feature_importance.to_excel(
        f'../../5_results/5_3_mix/feature_importance_set_{i}.xlsx',
        index=True
    )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       8.96 GB / 15.94 GB (56.2%)
Disk Space Avail:   79.11 GB / 446.36 GB (17.7%)
Presets specified: ['high_quality']


Fitting model 0
Original data: 223 59
Synthetic data: 248 248
Sampled data: 84
Train data: ED_2Clases
H    307
D    307
Name: count, dtype: int64


Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "c:\Users\jgala\uned\tfm\synthetic-data\4_ml\4_3_mix\AutogluonModels\mix\v0"
Train Data Rows:    614
Train Data Columns: 19
Label Column:       ED_2Clases
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = H, class 0 = D
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (H) vs negative (D) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Assigning sample weights to balance differences in frequency of classes.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9160.31 MB
	Train Da

[1000]	valid_set's binary_logloss: 0.300532
[2000]	valid_set's binary_logloss: 0.256584
[3000]	valid_set's binary_logloss: 0.237004
[4000]	valid_set's binary_logloss: 0.232351
[5000]	valid_set's binary_logloss: 0.229948


	0.9697	 = Validation score   (roc_auc)
	2.29s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch_r22 ... Training model for up to 7173.70s of the 7173.70s of remaining time.
	0.9258	 = Validation score   (roc_auc)
	2.54s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: XGBoost_r33 ... Training model for up to 7171.15s of the 7171.14s of remaining time.
	0.9697	 = Validation score   (roc_auc)
	0.84s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTrees_r42 ... Training model for up to 7170.28s of the 7170.28s of remaining time.
	0.9463	 = Validation score   (roc_auc)
	0.61s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost_r137 ... Training model for up to 7169.58s of the 7169.58s of remaining time.
	0.9624	 = Validation score   (roc_auc)
	1.04s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI_r102 ... Training model for up to 7168.54s of the 7168.53s of remainin

...

	0.9814	 = Validation score   (roc_auc)
	1.56s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost_r49 ... Training model for up to 6979.23s of the 6979.22s of remaining time.
	0.9925	 = Validation score   (roc_auc)
	5.87s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: CatBoost_r5 ... Training model for up to 6973.22s of the 6973.20s of remaining time.
	0.9911	 = Validation score   (roc_auc)
	5.66s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch_r87 ... Training model for up to 6967.54s of the 6967.53s of remaining time.
	0.9586	 = Validation score   (roc_auc)
	1.07s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch_r71 ... Training model for up to 6966.46s of the 6966.45s of remaining time.
	0.9754	 = Validation score   (roc_auc)
	3.06s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: CatBoost_r143 ... Training model for up to 6963.38s of the 6963.37s of remaining t

[1000]	valid_set's binary_logloss: 0.135492


	0.984	 = Validation score   (roc_auc)
	0.72s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: XGBoost_r22 ... Training model for up to 6946.41s of the 6946.40s of remaining time.
	0.9898	 = Validation score   (roc_auc)
	5.8s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetFastAI_r69 ... Training model for up to 6940.47s of the 6940.46s of remaining time.
	0.9733	 = Validation score   (roc_auc)
	0.63s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: CatBoost_r6 ... Training model for up to 6939.82s of the 6939.81s of remaining time.
	0.9941	 = Validation score   (roc_auc)
	1.42s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI_r138 ... Training model for up to 6938.39s of the 6938.38s of remaining time.
No improvement since epoch 6: early stopping
	0.962	 = Validation score   (roc_auc)
	0.93s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_r121 ... Training model for up

[1000]	valid_set's binary_logloss: 0.146321


No improvement since epoch 1: early stopping
	0.958	 = Validation score   (roc_auc)
	2.04s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: CatBoost_r180 ... Training model for up to 6934.40s of the 6934.39s of remaining time.
	0.9902	 = Validation score   (roc_auc)
	4.31s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch_r76 ... Training model for up to 6930.09s of the 6930.08s of remaining time.
	0.9788	 = Validation score   (roc_auc)
	2.48s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTrees_r197 ... Training model for up to 6927.59s of the 6927.57s of remaining time.
	0.981	 = Validation score   (roc_auc)
	0.77s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetTorch_r121 ... Training model for up to 6926.74s of the 6926.73s of remaining time.
	0.9705	 = Validation score   (roc_auc)
	6.2s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI_r127 ... Training m